### <center> Calculating GSD from UAV image EXIF data using [PyExifTool](https://pypi.org/project/PyExifTool/) </center>
##### Requires Phil Harvey's [ExifTool](https://exiftool.org/) be installed and on system PATH


In [1]:
import os
import subprocess
import exiftool
import pandas as pd

# Check if ExifTool is accessible
try:
    subprocess.run(["exiftool", "-ver"], check=True)
    print("ExifTool is accessible.")
except subprocess.CalledProcessError:
    print("ExifTool is not accessible.")
print()

def GSD_from_exif_batch(folder):
    """
    Calculate ground sampling distance (GSD) in cm/pixel from Exif data for all images in folder
    :param folder: path to folder containing images
    :return: csv file containing Exif data and GSD for all images in folder
    """
    # Get list of .JPG,.PNG, or .TIF files in folder
    files = [file for file in os.listdir(folder) if file.endswith((".JPG", ".PNG", ".TIF"))]

    # Add folder path to each file and store as string in quotations
    files = [os.path.join(folder, file) for file in files]

    # Print all tags for first file in folder
    with exiftool.ExifToolHelper() as et:
        for d in et.get_metadata(files[0]):
            for k, v in d.items():
                print(f"{k} = {v}")
    print()

    # Empty list for storing Exif data
    Exif_data = []

    # Values for calculating ground sampling distance (GSD) W,H in cm/pixel are: SensorWidth (mm),
    # SensorHeight (mm), RelativeAltitude (m), FocalLength (mm), ImageWidth (px), ImageHeight (px)

    # Store specified tags as list of dictionaries for all files in folder
    with exiftool.ExifToolHelper() as et:
        for d in et.get_tags(files[0:-1], tags=["SourceFile", "FileName", "ImageWidth", "ImageHeight", # specify tags here
                                                "FocalLength", "RelativeAltitude", "GimbalRollDegree", "GimbalYawDegree",
                                                "GimbalPitchDegree", "FlightRollDegree", "FlightYawDegree", "FlightPitchDegree",
                                                "XMP:GPSLatitude", "XMP:GPSLongitude"]):
            # append d to Exif_data list
            Exif_data.append(d)

    # Exif_data is a list of dictionaries. Convert to pandas dataframe where each unique value for SourceFile is a row
    df = pd.DataFrame(Exif_data)

    # Clean up column names; remove the text before the colon
    df.columns = df.columns.str.split(":").str[-1]

    # remove plus sign from RelativeAltitude
    df["RelativeAltitude"] = df["RelativeAltitude"].str.replace("+", "")

    # Convert columns to float
    df["RelativeAltitude"] = df["RelativeAltitude"].astype(float)
    df["FocalLength"] = df["FocalLength"].astype(float)
    df["ImageWidth"] = df["ImageWidth"].astype(float)
    df["ImageHeight"] = df["ImageHeight"].astype(float)

    # Add sensor width to dataframe (UAV is DJI Mavic 3M)
    # https://sdk-forum.dji.net/hc/en-us/articles/12325496609689-What-is-the-custom-camera-parameters-for-Mavic-3-Enterprise-series-and-Mavic-3M-
    df["SensorWidth"] = 17.4

    # Add GSDw and to dataframe
    # GSDw (at nadir) = (SensorWidth * RelativeHeight * 100) / (FocalLength * ImageWidth)
    df["GSDw"] = (df.SensorWidth * df.RelativeAltitude * 100) / (df.FocalLength * df.ImageWidth)

    # Save dataframe to csv
    df.to_csv(os.path.join(folder, f"Exif_data_{df['GSDw'].mean():.2f}.csv"), index=False) # save mean GSDw in filename

    return df

ExifTool is accessible.



In [ ]:
# Define paths to folders containing images
folders = ["S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230823_Route9-Orchard1perp_0.39GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard1perp_0.84GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard2_0.86GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard3-closePar_RGB_0.67GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231046_004_Route9Orchard1South_0.75GSD/",
           "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231102_005_Route9Orchard1North_1.60GSD/"]

# Call function for each folder in folders
for folder in folders:
    GSD_from_exif_batch(folder) # saves csv file with GSD in folder


In [2]:
# load csv "S:\Zack\Imagery\Chestnut\Best_FlightParameters\best_images.csv" as pd dataframe
df = pd.read_csv("S:/Zack/Imagery/Chestnut/Best_FlightParameters/best_images.csv")

# df.set_index("Tree ID", inplace=True)

In [3]:
dir = "S:/Zack/Imagery/Chestnut/Best_FlightParameters/"

# create folder in dir called bestImages_PNGs
if not os.path.exists(os.path.join(dir, "bestImages_PNGs")):
    os.makedirs(os.path.join(dir, "bestImages_PNGs"))

In [ ]:
# import shutil

# for treeID in df['Tree ID']:
#     # copy image from source to destination
#     src = f"S:/Zack/Imagery/Chestnut/Individual_Tree_Trunks/Route9/Route9_Orchard3_DawsonLab/easyIDP/Roboflow/images/{treeID}.png"
#     dst = f"S:/Zack/Imagery/Chestnut/Best_FlightParameters/bestImages_PNGs/{treeID}.png"
#     shutil.copy(src, dst)

In [4]:
# create folder in dir called bestImages_raw
if not os.path.exists(os.path.join(dir, "bestImages_raw")):
    os.makedirs(os.path.join(dir, "bestImages_raw"))

In [5]:
# make list of treeIDs in bestImages_PNGs folder
treeIDs = [file.split(".")[0] for file in os.listdir(os.path.join(dir, "bestImages_PNGs"))]

In [6]:
import shutil

# for each image in bestImages_PNGs folder, copy the corresponding raw image to bestImages_raw folder. 
# use df to get path of raw image, and copy from path to bestImages_raw folder
for treeID in treeIDs:
    # get path of raw image
    raw = df.loc[int(treeID), "Best UAV Image Path"]
    # copy path to bestImages_raw folder
    shutil.copy(raw, os.path.join(dir, "bestImages_raw"))


In [7]:
best_image_folder = "S:/Zack/Imagery/Chestnut/Best_FlightParameters/bestImages_raw/"

In [8]:
exif_df = GSD_from_exif_batch(best_image_folder)

SourceFile = S:/Zack/Imagery/Chestnut/Best_FlightParameters/bestImages_raw/DJI_20230823123355_0083_D.JPG
ExifTool:ExifToolVersion = 12.7
File:FileName = DJI_20230823123355_0083_D.JPG
File:Directory = S:/Zack/Imagery/Chestnut/Best_FlightParameters/bestImages_raw
File:FileSize = 10399744
File:FileModifyDate = 2024:04:15 12:06:52-05:00
File:FileAccessDate = 2024:04:15 12:07:00-05:00
File:FileCreateDate = 2024:04:15 12:06:52-05:00
File:FilePermissions = 100666
File:FileType = JPEG
File:FileTypeExtension = JPG
File:MIMEType = image/jpeg
File:ExifByteOrder = II
File:ImageWidth = 5280
File:ImageHeight = 3956
File:EncodingProcess = 0
File:BitsPerSample = 8
File:ColorComponents = 3
File:YCbCrSubSampling = 2 2
EXIF:ImageDescription = default
EXIF:Make = DJI
EXIF:Model = M3M
EXIF:Orientation = 1
EXIF:XResolution = 72
EXIF:YResolution = 72
EXIF:ResolutionUnit = 2
EXIF:Software = 10.12.05.45
EXIF:ModifyDate = 2023:08:23 12:33:55
EXIF:YCbCrPositioning = 2
EXIF:ExposureTime = 0.001
EXIF:FNumber = 2.8

In [9]:
# change Best UAV Image Path in df to image name. 
df["Best UAV Image Path"] = df["Best UAV Image Path"].str.split("/").str[-1]

In [10]:
# change column name to "image Name"
df.rename(columns={"Best UAV Image Path": "Image Name"}, inplace=True)

In [11]:
# if df['Image Name'] in exif_df['FileName'], then join the two dataframes using the Image Name as the key
df_merged = pd.merge(df, exif_df, left_on="Image Name", right_on="FileName")

In [12]:
# remove duplicate rows
df_merged.drop_duplicates(inplace=True)

In [13]:
# subtract 1 from tree ID
df_merged["Tree ID"] = df_merged["Tree ID"] - 1

In [14]:
print(df_merged)

    Tree ID                     Image Name  \
0        14  DJI_20230823131758_0620_D.JPG   
1        28  DJI_20230823131045_0338_D.JPG   
2        32  DJI_20230823125646_0967_D.JPG   
3        33  DJI_20230823132018_0713_D.JPG   
4        35  DJI_20230823125647_0968_D.JPG   
..      ...                            ...   
93      393  DJI_20230823123539_0152_D.JPG   
94      406  DJI_20230823124307_0447_D.JPG   
95      407  DJI_20230823124047_0354_D.JPG   
96      408  DJI_20230823130933_0291_D.JPG   
97      410  DJI_20230823130742_0219_D.JPG   

                                           SourceFile  \
0   S:/Zack/Imagery/Chestnut/Best_FlightParameters...   
1   S:/Zack/Imagery/Chestnut/Best_FlightParameters...   
2   S:/Zack/Imagery/Chestnut/Best_FlightParameters...   
3   S:/Zack/Imagery/Chestnut/Best_FlightParameters...   
4   S:/Zack/Imagery/Chestnut/Best_FlightParameters...   
..                                                ...   
93  S:/Zack/Imagery/Chestnut/Best_FlightParamete

In [15]:
# save df_merged to csv in dir
df_merged.to_csv(os.path.join(dir, "best_flightParameters.csv"), index=False) # save mean GSDw in filename